In [1]:
class RUNE:
    def __init__(self,mana=0,next_rune=None,location=None):
        self.mana=mana
        self.next_rune=next_rune
        self.location=location
    
    def effect_world(self, world):
        print("magic")
    
    def represent_thing(self, obj):
        return True
    
    def modify_spell(self, rune):
        print("arcane")
    
    def magic(self):
        self.mana-=1
        if self.mana<=0:
            print("failure")
            return None
        
        if self.next_rune is None:
            self.effect_world(self.location)
        
        if isinstance(self.next_rune,RUNE):
            self.modify_spell(self.next_rune)
            self.next_rune.magic()
            
        return None
        
    

In [2]:
class PUSH(RUNE):
    def effect_world(self, world):
        print("push")
    
    def represent_thing(self, obj):
        return True
    
    def modify_spell(self, rune):
        print("far")

In [3]:
class FIRE(RUNE):
    def effect_world(self, world):
        print("grill")
    
    def represent_thing(self, obj):
        return True
    
    def modify_spell(self, rune):
        print("hot")

In [4]:
runa=PUSH(4)
runa.magic()

push


In [5]:
runa=PUSH(4,FIRE(1))
runa.magic()

far
failure


In [6]:
runa=PUSH(4,FIRE(3))
runa.magic()

far
grill


In [7]:
runa=PUSH(4,FIRE(4,RUNE(2)))
runa.magic()

far
hot
magic


In [8]:
runa=RUNE(4)
runa.magic()

magic


In [9]:
"_@_**_"

'_@_**_'

# testing hexagons

In [1]:
# Import the pygame module
import pygame
import random
import copy

# Import pygame.locals for easier access to key coordinates
# Updated to conform to flake8 and black standards
from pygame.locals import (
    RLEACCEL,
    MOUSEBUTTONDOWN,
    MOUSEBUTTONUP,
    K_UP,
    K_s,
    K_r,
    K_m,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    K_SPACE,
    KEYDOWN,
    QUIT,
)


# Define constants for the screen width and height
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

pygame 2.1.2 (SDL 2.0.16, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Define a player object by extending pygame.sprite.Sprite
# The surface drawn on the screen is now an attribute of 'player'
class Mage(pygame.sprite.Sprite):
    def __init__(self):
        super(Mage, self).__init__()
        self.surf = pygame.image.load("assets/WizardSprite.png").convert_alpha()
        self.surf.set_colorkey(None, RLEACCEL)
        self.mask=pygame.mask.from_surface(self.surf)
        self.rect = self.surf.get_rect()
            
    def set_position(self,position):
        self.rect = self.surf.get_rect(center=position)

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 23 13:50:07 2022
@author: richa
"""
import random
from typing import List
from typing import Tuple

import pygame
from hexagon import FlatTopHexagonTile
from hexagon import HexagonTile

# pylint: disable=no-member


def create_hexagon(position, radius=50, flat_top=False) -> HexagonTile:
    """Creates a hexagon tile at the specified position"""
    class_ = FlatTopHexagonTile if flat_top else HexagonTile
    return class_(radius, position, colour=get_random_colour())


def get_random_colour(min_=150, max_=151) -> Tuple[int, ...]:
    """Returns a random RGB colour with each component between min_ and max_"""
    return tuple(random.choices(list(range(min_, max_)), k=3))


def init_hexagons(num_x=9, num_y=6, flat_top=False) -> List[HexagonTile]:
    """Creates a hexaogonal tile map of size num_x * num_y"""
    # pylint: disable=invalid-name
    leftmost_hexagon = create_hexagon(position=(30, 10), flat_top=flat_top)
    hexagons = [leftmost_hexagon]
    for x in range(num_y):
        if x:
            # alternate between bottom left and bottom right vertices of hexagon above
            index = 2 if x % 2 == 1 or flat_top else 4
            position = leftmost_hexagon.vertices[index]
            leftmost_hexagon = create_hexagon(position, flat_top=flat_top)
            hexagons.append(leftmost_hexagon)

        # place hexagons to the left of leftmost hexagon, with equal y-values.
        hexagon = leftmost_hexagon
        for i in range(num_x):
            x, y = hexagon.position  # type: ignore
            if flat_top:
                if i % 2 == 1:
                    position = (x + hexagon.radius * 3 / 2, y - hexagon.minimal_radius)
                else:
                    position = (x + hexagon.radius * 3 / 2, y + hexagon.minimal_radius)
            else:
                position = (x + hexagon.minimal_radius * 2, y)
            hexagon = create_hexagon(position, flat_top=flat_top)
            hexagons.append(hexagon)

    return hexagons


def render_hexagons(screen, hexagons):
    """Renders hexagons on the screen"""
    
    #draw hexagons
    for hexagon in hexagons:
        hexagon.render(screen)
        hexagon.render_highlight(screen, border_colour=(50, 50, 50))

    # draw borders around colliding hexagons and neighbours
    mouse_pos = pygame.mouse.get_pos()
    colliding_hexagons = [
        hexagon for hexagon in hexagons if hexagon.collide_with_point(mouse_pos)
    ]
    for hexagon in colliding_hexagons:
        for neighbour in hexagon.compute_neighbours(hexagons):
            neighbour.render_highlight(screen, border_colour=(50, 50, 50))
        hexagon.render_highlight(screen, border_colour=(255, 0, 0))

In [ ]:
def main():
    """Main function"""
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    hexagons = init_hexagons(flat_top=True)
    mage=Mage()
    mage.set_position(hexagons[25].centre)
    running = True
    
    while running:
        # for loop through the event queue
        for event in pygame.event.get():

            # Check for KEYDOWN event
            if event.type == KEYDOWN:
                # If the Esc key is pressed, then exit the main loop
                if event.key == K_ESCAPE:
                    running = False

            # Check for QUIT event. If QUIT, then set running to false.
            elif event.type == QUIT:
                running = False

            if event.type == MOUSEBUTTONDOWN:
                # get a list of all sprites that are under the mouse cursor
                pos = pygame.mouse.get_pos()
                print(pos)
                
        for hexagon in hexagons:
            hexagon.update()

        #drawing
        screen.fill((0, 0, 0))
        #draw hexagons
        render_hexagons(screen, hexagons)
        #draw mage
        screen.blit(mage.surf, mage.rect)
        pygame.display.flip()
        #FPS
        clock.tick(30)
        
    pygame.display.quit()


if __name__ == "__main__":
    main()